# SQL Leetcode 50

## Basic 1-5

In [ ]:
Select  product_id 
from Products
where low_fats = 'Y'  and recyclable = 'Y'

In [ ]:
select name
from Customer
where referee_id != 2 or referee_id is null

In [ ]:
Select name
, population
, area 
from World
where area >= 3000000 
or population >= 25000000

In [ ]:
select distinct author_id as id 
from views
where author_id = viewer_id 
order by author_id asc

In [ ]:
select tweet_id 
from Tweets
where length(content) >15

## Basic Join 6 - 14

In [ ]:
select 
     e2.unique_id
    , e1.name     
from Employees e1
left join EmployeeUNI e2
on e1.id = e2.id

In [ ]:
select 
p.product_name
, s.year 
, s.price 
from sales s
left join Product p
on s.product_id = p.product_id

In [ ]:
with left_join as (
SELECT 
    v.customer_id
    , t.transaction_id
FROM Visits v
LEFT JOIN Transactions t
ON v.visit_id = t.visit_id
WHERE t.transaction_id IS NULL
)

select customer_id
, count(*) as count_no_trans
from left_join
group by customer_id

1. **第一个步骤**：首先，查询使用 `LEFT JOIN` 将 `Visits` 表和 `Transactions` 表联接在一起，联接条件是 `visit_id`。`LEFT JOIN` 会返回所有 `Visits` 表中的行，以及与 `Transactions` 表中匹配的行。如果在 `Transactions` 表中找不到匹配的行，那么 `transaction_id` 的值就会是 `NULL`。查询中的 `WHERE` 子句 `t.transaction_id IS NULL` 会过滤出那些没有匹配的行，也就是那些没有交易的访问。
2. **第二个步骤**：接着，查询对第一个步骤的结果按 `customer_id` 分组，并计算每个组的行数。这个行数就是每个客户没有交易的次数。

In [ ]:
WITH date_pairs AS (
    SELECT 
        w1.id as id,
        w1.recordDate as date1, 
        w2.recordDate as date2, 
        w1.temperature as temp1, 
        w2.temperature as temp2
    FROM Weather w1, Weather w2 
    Order by w1.id
)

SELECT id
FROM date_pairs
WHERE DATEDIFF(date1, date2) = 1 
    AND temp1 > temp2

In [ ]:
with timev as (
select machine_id
, process_id
, activity_type
, timestamp - LEAD(timestamp) OVER(PARTITION BY machine_id, process_id ORDER BY activity_type DESC) AS difference
from Activity
 )

SELECT
  machine_id
  , ROUND(AVG(difference), 3) AS processing_time
FROM timev 
GROUP BY 1

In [ ]:
select 
e.name 
,b.bonus 
from employee e
left join bonus b
on e.empId = b.empId 
where b.bonus < 1000 
or b.bonus is null

In [ ]:
WITH ExaminationsCount AS (
    SELECT student_id
    , subject_name
    , COUNT(*) AS attended_exams
    FROM Examinations
    GROUP BY student_id, subject_name
 )

SELECT 
    s.student_id,
    s.student_name, 
    sub.subject_name, 
    COALESCE(e.attended_exams, 0) as attended_exams
FROM students s
CROSS JOIN Subjects sub
LEFT JOIN ExaminationsCount e
ON s.student_id = e.student_id AND sub.subject_name = e.subject_name
ORDER BY s.student_id, sub.subject_name

In [ ]:
select 
e1.name
from Employee e1
join Employee e2
on e1.id = e2.managerId
group by e2.managerId
having count(e2.id)>=5

In [ ]:
With confirmed as (
    SELECT user_id, 
    IF(action="confirmed",1,0) AS is_confirmed
    FROM Confirmations
)
,average as (
 SELECT user_id, 
    ROUND(AVG(is_confirmed),2) AS confirmation_rate
  FROM confirmed
  GROUP BY user_id
)

SELECT s.user_id, 
       COALESCE(av.confirmation_rate, 0) AS confirmation_rate
FROM Signups AS s 
LEFT JOIN average AS av
ON s.user_id = av.user_id;

## Basic Aggregate Functions 15

In [ ]:
select id
, movie 
, description
, rating
from Cinema
where (id % 2) > 0
and description != 'boring' 
order by rating desc

In [ ]:
with ori as (
select 
p.product_id
, p.price 
, s.units
from Prices p
join UnitsSold s
on p.product_id = s.product_id
AND s.purchase_date BETWEEN p.Start_date and p.end_date
)

select
product_id
, round(sum(price * units) / sum(units),2) as average_price
from ori
group by p.product_id

In [ ]:
with joins as (
select p.project_id 
, p.employee_id 
, e.name 
, e.experience_years
from Project p
join Employee e
on p.employee_id = e.employee_id
)

select project_id
     , round(avg(experience_years),2) as average_years
from joins
group by 1

In [ ]:
WITH total_users AS (
  SELECT COUNT(user_id) as total 
  FROM Users
)
,
contest_users AS (
  SELECT 
    r.contest_id, 
    count(distinct r.user_id) as user_count
  FROM Register r
  GROUP BY r.contest_id
)

SELECT 
  cu.contest_id, 
  ROUND((cu.user_count / tu.total) * 100, 2) as percentage
FROM contest_users cu, total_users tu
ORDER BY percentage DESC, cu.contest_id;

In [ ]:
select query_name
, round(avg(rating/position),2) as quality
, round(sum(rating < 3)/ count(query_name)*100,2) as poor_query_percentage
from Queries
group by query_name

In [ ]:
SELECT 
SUBSTR(trans_date,1,7) as month
, country
, count(*) as trans_count
, SUM(IF(state = 'approved', 1, 0)) as approved_count
, SUM(amount) as trans_total_amount
, SUM(IF(state = 'approved',amount,0)) as approved_total_amount

FROM Transactions
GROUP BY SUBSTR(trans_date,1,7), country

In [ ]:
WITH table1 AS (
SELECT
  customer_id
  , min(order_date) as first_order
FROM Delivery 
GROUP BY customer_id
 )

SELECT 
ROUND(SUM(IF(d.order_date = d.customer_pref_delivery_date, 1, 0)) / COUNT(*) * 100, 2) as immediate_percentage 
FROM Delivery d
INNER JOIN table1 t
ON d.customer_id = t.customer_id 
AND d.order_date = t.first_order

In [ ]:
with first as(
select 
player_id
, min(event_date) as first_log
from Activity
group by player_id
)
, later_log as(
select 
a.player_id
from Activity a
join first f
on a.player_id = f.player_id
where a.event_date = DATE_ADD(f.first_log, INTERVAL 1 DAY)
)

select 
round(count(distinct l.player_id) 
  / count(distinct f.player_id) ,2) as fraction
from first f
left join later_log l
on l.player_id = f.player_id

## Sorting and Grouping 23

In [ ]:
SELECT
teacher_id
, COUNT(DISTINCT subject_id) as cnt
FROM Teacher
GROUP BY teacher_id

In [ ]:
SELECT 
activity_date as day
, COUNT(DISTINCT user_id) as active_users
FROM Activity
WHERE (activity_date > "2019-06-27" 
AND activity_date <= "2019-07-27")
GROUP BY activity_date

In [ ]:
WITH product_first_year AS (
    SELECT product_id, MIN(year) AS first_year
    FROM Sales 
    GROUP BY product_id
)
SELECT s.product_id
, s.year AS first_year
, s.quantity
, s.price
FROM Sales s
JOIN product_first_year pfy 
ON s.product_id = pfy.product_id 
AND s.year = pfy.first_year;

In [ ]:
SELECT class
FROM Courses
GROUP BY class
HAVING COUNT(class) >=5

In [ ]:
SELECT user_id
, COUNT(follower_id) AS followers_count
FROM Followers
GROUP BY user_id
ORDER BY user_id

In [ ]:
WITH ONCE AS(
SELECT num
FROM MyNumbers
GROUP BY num
HAVING COUNT(num) = 1
)
SELECT MAX(M.num) AS num
FROM MyNumbers M
JOIN ONCE O
ON O.num = M.num

In [ ]:
WITH customer_sum AS (
    SELECT customer_id, SUM(DISTINCT product_key) AS sum_product_key
    FROM Customer
    GROUP BY customer_id
),
total_product_sum AS (
    SELECT SUM(product_key) AS total_product_key
    FROM Product
)
SELECT cs.customer_id
FROM customer_sum cs, total_product_sum tps
WHERE cs.sum_product_key = tps.total_product_key;

## Advanced Select and Joins 30

In [ ]:
with direct_reports AS (
  SELECT
    reports_to AS manager_id 
    , COUNT(DISTINCT employee_id) AS reports_count
    , ROUND(AVG(age), 0) AS average_age
  FROM Employees
  GROUP BY 1
)

SELECT 
  e.employee_id
  , e.name
  , dr.reports_count
  , dr.average_age
from Employees e
left join direct_reports dr 
  ON e.employee_id = dr.manager_id
  where reports_count > 0 
Order by employee_id

In [ ]:
select employee_id,
case when count(employee_id) = 1 then department_id
else max(if(primary_flag = 'Y',department_id, 0))
end as department_id
from Employee
group by employee_id

In [ ]:
SELECT 
*
, CASE WHEN 
x + y > z and x + z > y and y + z > x  THEN 'Yes'
ELSE 'No'
END AS triangle
FROM Triangle

In [ ]:
SELECT distinct 
    i1.num as ConsecutiveNums 
FROM 
    logs i1,
    logs i2,
    logs i3
WHERE 
    i1.id=i2.id+1 AND 
    i2.id=i3.id+1 AND 
    i1.num=i2.num AND 
    i2.num=i3.num

In [ ]:
with prod as (
    select 
            product_id
            , max(change_date) as ldate  
    from Products 
    where change_date <= '2019-08-16' 
    group by product_id
    )

select distinct p.product_id
-- 如果一个产品的新价格是 NULL（这意味着这个产品在查询日期之前没有价格变更），那么它的价格就是 10。
, coalesce(pri.new_price,10) as price 
from Products p 
left join prod pr
on pr.product_id=p.product_id 
left join Products pri 
on pri.product_id=p.product_id 
and pri.change_date=pr.ldate

1. 使用 CTE 找出每个产品的最后更改日期（如果存在）。
2. 将 **`Products`** 表和 CTE 连接起来，获取每个产品的最后更改日期。
3. 再次将结果和 **`Products`** 表连接起来，获取每个产品的新价格。

In [ ]:
SELECT q1.person_name
FROM Queue q1 
JOIN Queue q2 
ON q1.turn >= q2.turn
GROUP BY q1.turn
HAVING SUM(q2.weight) <= 1000
ORDER BY SUM(q2.weight) DESC
LIMIT 1

这个问题的解题思路是利用 SQL 的 `JOIN`、`GROUP BY` 和 `HAVING` 来计算每个人以及他们前面的所有人的体重总和，然后找出体重总和不超过 1000 公斤的最后一个人。

1. `FROM Queue q1 JOIN Queue q2 ON q1.turn >= q2.turn`：这个 `JOIN` 的目的是为了获取每个人（`q1`）和他们前面的所有人（`q2`）。如果 `q1.turn` 大于等于 `q2.turn`，那么 `q2` 就是 `q1` 前面的一个人。这个 `JOIN` 会返回所有这样的配对。
2. `GROUP BY q1.turn`：这个 `GROUP BY` 的目的是将上一步返回的配对按照 `q1` 的 `turn` 分组。每个组代表一个人和他前面的所有人。
3. `HAVING SUM(q2.weight) <= 1000`：这个 `HAVING` 子句的目的是过滤掉体重总和超过 1000 公斤的组。`SUM(q2.weight)` 计算的是每个组中所有人的体重总和。
4. `ORDER BY SUM(q2.weight) DESC`：这个 `ORDER BY` 子句的目的是按照体重总和的降序排列组。因为我们想找的是体重总和不超过 1000 公斤的最后一个人，所以我们需要先看体重最重的组。
5. `LIMIT 1`：这个 `LIMIT` 子句的目的是只返回排名最高的一行，也就是体重总和不超过 1000 公斤的最后一个人。

总的来说，这个查询的目的是找出可以登上公交车的最后一个人，也就是他和他前面的所有人的体重总和不超过 1000 公斤的最后一个人。

In [ ]:
select 'Low Salary' as category
, count(*) as accounts_count
from Accounts
where income < 20000

UNION 
select 'High Salary', count(*)
from Accounts 
where income > 50000

UNION
select 'Average Salary', count(*)
from Accounts 
where income >= 20000 and income <= 50000

这个 SQL 查询的目的是计算每个薪资等级的银行账户数量。查询的主要部分是三个 `SELECT` 语句，它们分别计算了 "Low Salary"、"High Salary" 和 "Average Salary" 这三个薪资等级的账户数量。这三个 `SELECT` 语句通过 `UNION` 连接在一起，形成了最终的结果。

让我们逐个解析这三个 `SELECT` 语句：

1. **第一个 `SELECT` 语句**：这个语句选取了所有薪资严格小于 20000 的账户，将它们的薪资等级设为 "Low Salary"，并计算了这些账户的数量。
2. **第二个 `SELECT` 语句**：这个语句选取了所有薪资严格大于 50000 的账户，将它们的薪资等级设为 "High Salary"，并计算了这些账户的数量。
3. **第三个 `SELECT` 语句**：这个语句选取了所有薪资在 20000 和 50000（包含两端）之间的账户，将它们的薪资等级设为 "Average Salary"，并计算了这些账户的数量。

最后，通过 `UNION` 将这三个 `SELECT` 语句的结果合并在一起，得到了每个薪资等级的银行账户数量。注意，`UNION` 会自动去除重复的行，但是在这个查询中，由于每个 `SELECT` 语句的结果的薪资等级都是不同的，所以不会有重复的行。

## Subqueries 37

In [ ]:
WITH salary AS(
SELECT *
FROM Employees
WHERE salary < 30000
)

SELECT employee_id
FROM salary
WHERE manager_id NOT IN (
  SELECT employee_id
  FROM Employees
  )
ORDER BY employee_id

In [ ]:
SELECT 
case 
    when id % 2 = 0 then id - 1
    when id % 2 <> 0 and id = (select count(*) from Seat ) then id
    else id +1
    end as id
, student
FROM Seat
ORDER BY id

In [ ]:
(
SELECT 
u.name as results
FROM Users u
LEFT JOIN MovieRating R
ON U.user_id = R.user_id
group by r.user_id
order by count(movie_id) desc , u.name
LIMIT 1
)

union all

(
SELECT
m.title as results
FROM Movies m 
left join MovieRating r
on m.movie_id = r.movie_id
where DATE(r.created_at) BETWEEN '2020-02-01' AND '2020-02-29'
group by m.title 
order by avg(r.rating) desc, m.title
limit 1
)

In [ ]:
WITH daily_amounts AS (
    SELECT 
        visited_on, 
        SUM(amount) as total_amount
    FROM customer 
    GROUP BY 1
),
windowed AS (
    SELECT 
        visited_on
        , COUNT(visited_on) OVER (
            ORDER BY visited_on ROWS BETWEEN 6 PRECEDING and CURRENT ROW
        ) as day_cnt
        , SUM(total_amount) OVER (
            ORDER BY visited_on ROWS BETWEEN 6 PRECEDING and CURRENT ROW
        ) as amount
        , AVG(total_amount) OVER (
            ORDER BY visited_on ROWS BETWEEN 6 PRECEDING and CURRENT ROW
        ) as average_amount
    FROM daily_amounts
)
SELECT 
    visited_on
    , amount
    , ROUND(average_amount, 2) AS average_amount
FROM  windowed
WHERE  day_cnt = 7

In [ ]:
with united as
(   
    select requester_id
    from RequestAccepted

    union all

    select accepter_id
    from RequestAccepted
)

select 
requester_id as id
, count(*) as num
from united
group by 1
order by 2 desc
limit 1

In [ ]:
WITH VAL AS (
SELECT 
tiv_2015
FROM Insurance
GROUP BY tiv_2015
HAVING COUNT(*) > 1 
),
LOCATION AS (
SELECT 
lat
, lon
FROM Insurance
GROUP BY 1,2
HAVING COUNT(*) = 1
)

SELECT 
round(sum(tiv_2016),2) as tiv_2016
FROM Insurance
WHERE tiv_2015 IN (SELECT tiv_2015 FROM VAL )
AND (lat,lon) IN (SELECT lat, lon FROM LOCATION)

In [ ]:
with salary_rank as(
    SELECT 
        *, 
        DENSE_RANK() OVER(PARTITION BY departmentId ORDER BY salary DESC) AS salary_ranking
    FROM Employee
    )

, organized as (
    SELECT 
    departmentId
    , name
    , salary
    FROM salary_rank
    WHERE salary_ranking <= 3
)

select 
d.name as Department
, o.name as Employee
, salary 
from organized o
left join Department d
on d.id = o.departmentId
order by salary

## Advanced String Functions / Regex / Clause 44

In [ ]:
select 
user_id
, CONCAT(UPPER(SUBSTRING(name, 1, 1)), LOWER(SUBSTRING(name, 2))) AS name
FROM Users
ORDER BY user_id

In [ ]:
SELECT *
FROM Patients
WHERE conditions LIKE "% DIAB1%"
or conditions LIKE "DIAB1%"

In [ ]:
DELETE p1 FROM Person p1
INNER JOIN Person p2 
WHERE p1.email = p2.email AND p1.id > p2.id

In [ ]:
with ranking_all as (
select 
*
, dense_rank()over(order by salary desc) as ranking
from Employee
)

select 
max(salary) as SecondHighestSalary
from ranking_all
where ranking = 2

In [ ]:
SELECT 
sell_date
, COUNT(DISTINCT product) AS num_sold
, GROUP_CONCAT(distinct product order by product separator ',') AS products
FROM Activities
GROUP BY 1
ORDER BY 1

In [ ]:
select 
p.product_name
, sum(o.unit) as unit
from Products p
left join Orders o
on p.product_id = o.product_id
where DATE(order_date) BETWEEN '2020-02-01' AND '2020-02-29'
group by p.product_name
having sum(o.unit) >= 100

In [ ]:
SELECT *
FROM Users
WHERE mail REGEXP '^[a-zA-Z][a-zA-Z0-9_.-]*@leetcode.com$'
AND mail NOT LIKE '%?%'